In [ ]:
import numpy as np

In [ ]:
N = 6
S = 8
I1 = 5
I2 = 4

alpha_1 = 0.1
alpha_2 = 0.01

tau = 1 / (1 + I1)
p1 = (alpha_1 ** (1 - tau)) * (alpha_2 ** tau)
p2 = 5 * p1 / np.sqrt(S)

print(f"""
N = {N}
S = {S}
I1 = {I1}
I2 = {I2}
alpha_1 = {alpha_1}
alpha_2 = {alpha_2}
tau = {tau}
p1 = {p1}
p2 = {p2}
""")